## Data Fetching Examples

### 1- Import the DataFetcher

In [3]:
from torchtrade.data_processors.ccxt_fetcher import CcxtFetcher
from torchtrade.envs.components.market import  Market
from torchtrade.envs.components.clock import Clock
from torchtrade.trade.base import Trade ,TradeDirection 
from datetime import datetime
import pandas as pd
import numpy as np

In [4]:
# Setting params
exchange = "binance"
symbols = ["BTC/USDT","ETH/USDT"]
timeFrame = "1d"

In [5]:
# Defining the market
datafetcher = CcxtFetcher(exchange,symbols,timeFrame)
df = datafetcher.fetch_data()

Fetching 1d timeframe data since 2017-08-17 00:00:00 until 2023-02-22 00:00:00


In [6]:
#Setting the clock
time_start = df.index[0][1]
time_end = df.index[-1][1]


clock = Clock(time_start,timeFrame)

mkt = Market(df)

#Market is observing the clock
clock.register_observer(mkt)


In [7]:
df

isTraded     open     high      low    close       volume
symbol   timestamp                                                           
BTC/USDT 2017-08-17        1  4261.48  4485.39  4200.74  4285.08   795.150377
         2017-08-18        1  4285.08  4371.52  3938.77  4108.37  1199.888264
         2017-08-19        1  4108.37  4184.69   3850.0  4139.98   381.309763
         2017-08-20        1  4120.98  4211.08  4032.62  4086.29   467.083022
         2017-08-21        1  4069.13  4119.62  3911.79   4016.0    691.74306
...                      ...      ...      ...      ...      ...          ...
ETH/USDT 2023-02-18        1  1693.81  1713.13  1679.77  1691.62  267305.7813
         2023-02-19        1  1691.62  1726.59  1666.14  1679.75  401023.0477
         2023-02-20        1  1679.74  1719.89  1650.56  1703.27  451314.8335
         2023-02-21        1  1703.27  1716.66   1636.3  1659.76  520354.4688
         2023-02-22        1  1659.77  1666.95  1621.59   1643.9  250956.9619

[4032 rows x 6 columns]

Labeling

In [10]:
clock.reset()

stop_loss_levels = [1,2,3,4,5]
rr_levels = [1.5 , 2 , 2.5,3]
mkt.observers=[]

trades= []
while clock.timestamp < time_end : 
    print(clock.timestamp)
    print(len(mkt.observers))
    data = mkt.data
    for index, row in data.iterrows():
        if row['isTraded'] == 1:            
            trade = [[index[0],index[1],sl,rr] + [Trade("zak",index[1],index[0],clock.timeframe,TradeDirection.LONG,1,1,mkt, stop_loss=sl, risk_reward=rr)] for sl in stop_loss_levels for rr in rr_levels] 
            trades+=trade
    clock.next()

trd = pd.DataFrame(trades, columns =['symbol', 'timestamp','stopLoss','riskReward','trade'])
trd.set_index(['symbol', 'timestamp', 'stopLoss', 'riskReward'], inplace=True)
trd["win/loss"] = trd['trade'].apply(lambda t : t.reward)
trd.to_csv('data.csv',index=False)


2017-08-17 00:00:00


ValueError: No data found for symbol 2017-08-17 00:00:00

In [ ]:
trd.head(100)

trade  \
symbol   timestamp  stopLoss riskReward                                                      
BTC/USDT 2017-08-17 1        1.5         \n            Current Timestamp: 2023-02-20 00...   
                             2.0         \n            Current Timestamp: 2023-02-20 00...   
                             2.5         \n            Current Timestamp: 2023-02-20 00...   
                             3.0         \n            Current Timestamp: 2023-02-20 00...   
                    2        1.5         \n            Current Timestamp: 2023-02-20 00...   
...                                                                                    ...   
         2017-08-19 4        3.0         \n            Current Timestamp: 2023-02-20 00...   
                    5        1.5         \n            Current Timestamp: 2023-02-20 00...   
                             2.0         \n            Current Timestamp: 2023-02-20 00...   
                             2.5         \n            Current Timestamp: 2023-02-20 00...   
                             3.0         \n            Current Timestamp: 2023-02-20 00...   

                                         win/loss  
symbol   timestamp  stopLoss riskReward            
BTC/USDT 2017-08-17 1        1.5        -1.000000  
                             2.0        -1.000000  
                             2.5        -1.000000  
                             3.0        -1.000000  
                    2        1.5        -2.000000  
...                                           ...  
         2017-08-19 4        3.0        -4.000000  
                    5        1.5        -5.000000  
                             2.0         3.138106  
                             2.5        -5.000000  
                             3.0        -5.000000  

[100 rows x 2 columns]

In [ ]:

# Sort the DataFrame by "symbol", "timestamp", and "reward" in descending order
trd.reset_index(inplace=True)
tr = trd.sort_values(by=['symbol', 'timestamp', 'win/loss'], ascending=[True, True, False])

# Drop duplicates based on "symbol" and "timestamp", keeping only the first row for each combination
tr = tr.drop_duplicates(subset=['symbol', 'timestamp'])

# Select the desired columns
tr = tr[['symbol', 'timestamp', 'stopLoss', 'riskReward' ,'win/loss']]




ValueError: cannot insert level_0, already exists

In [ ]:
tr.to_csv('trades.csv',index=False)

In [ ]:
from torchtrade.trade.base import BaseTrade 


trade = BaseTrade()

TypeError: Can't instantiate abstract class BaseTrade with abstract methods entry_trigger, exit_trigger